In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/1016version.csv')
data = df.copy()

In [190]:
positions = ['SUPPORT', 'ADC', 'MIDDLE', 'JUNGLE', 'TOP']
t1_fields = []

for position in range(1, 6):
    field = f't1p{position}_champId'
    t1_fields.append(field)

t1 = df[t1_fields].copy()
t1['t1_win'] = df['t1_win']
t1 = t1.melt(id_vars=['t1_win'], value_vars=t1_fields[:], var_name='position', value_name='champId')
t1['position'] = t1['position'].replace(dict(zip(t1_fields, positions)))
t1 = t1.groupby(['position', 'champId']).agg({'t1_win': ['count', 'sum', 'mean']})
t1.columns = ['appearances', 'wins', 'winrate']
t1 = t1.reset_index()


In [191]:
t2_fields = []

for position in range(1, 6):
    field = f't2p{position}_champId'
    t2_fields.append(field)

t2 = df[t2_fields].copy()
t2['t2_win'] = df['t1_win'].replace({0: 1, 1: 0})
t2 = t2.melt(id_vars=['t2_win'], value_vars=t2_fields[:], var_name='position', value_name='champId')
t2['position'] = t2['position'].replace(dict(zip(t2_fields, positions)))
t2 = t2.groupby(['position', 'champId']).agg({'t2_win': ['count', 'sum', 'mean']})
t2.columns = ['appearances', 'wins', 'winrate']
t2 = t2.reset_index()

In [192]:
t2

,position,champId,appearances,wins,winrate
0,ADC,1,8,7,0.875000
1,ADC,2,14,9,0.642857
2,ADC,3,12,8,0.666667
3,ADC,7,1,0,0.000000
4,ADC,8,15,6,0.400000
...,...,...,...,...,...
590,TOP,518,288,142,0.493056
591,TOP,555,37,23,0.621622
592,TOP,777,659,319,0.484067
593,TOP,875,947,458,0.483633


In [193]:
t1

,position,champId,appearances,wins,winrate
0,ADC,1,18,8,0.444444
1,ADC,2,13,5,0.384615
2,ADC,3,7,2,0.285714
3,ADC,4,2,1,0.500000
4,ADC,7,2,1,0.500000
...,...,...,...,...,...
597,TOP,523,2,1,0.500000
598,TOP,555,41,24,0.585366
599,TOP,777,675,333,0.493333
600,TOP,875,990,492,0.496970


In [194]:
merged_table = pd.merge(t1, t2, on=['position', 'champId'], how='outer')
merged_table = merged_table.fillna(0)
merged_table['appearances'] = merged_table['appearances_x'].add(merged_table['appearances_y'])
merged_table['wins'] = merged_table['wins_x'].add(merged_table['wins_y'])
merged_table = merged_table.drop(columns=['appearances_x', 'appearances_y', 'wins_x', 'wins_y', 'winrate_x', 'winrate_y'])

In [195]:
merged_table['winrate'] = merged_table['wins'] / merged_table['appearances']

In [196]:
pivot_table = merged_table.pivot_table(index='champId', 
                                        columns='position', 
                                        values=['appearances', 'wins', 'winrate'],
                                        aggfunc='sum')

In [197]:
pivot_table.columns = ['_'.join(reversed(col)).strip() for col in pivot_table.columns.values]
pivot_table = pivot_table.sort_index(axis=1)

In [198]:
pivot_table = pivot_table.reset_index()

In [200]:
pivot_table.to_csv('data/table3.csv', index=False)

In [10]:
names = pd.read_csv('data/names.csv')

In [14]:
for t in range(1, 3):
    for position in range(1, 6):
        field = f't{t}p{position}_champId'
        df[field] = df[field].map(names.set_index('champId')['champName'])
        df = df.rename(columns={field: f't{t}p{position}_champName'})

In [16]:
df.to_csv('data/1016version.csv', index=False)

### Create a table of winrates for ml

In [3]:
winrates = pd.read_csv('data/winrates.csv')

In [4]:
fields = []
data_fields = []

for t in range(1, 3):
    for position in range(1, 6):
        field = f't{t}p{position}_champName'
        new_field = f't{t}p{position}_winrate'

        data[field] = df[field].map(winrates.set_index('champName')['winpercent'])
        data = data.rename(columns={field: new_field})

        fields.append(field)
        data_fields.append(new_field)



fields.append('t1_win')
data_fields.append('t1_win')


In [6]:
df[fields].to_csv('data/game_names.csv', index=False)

In [92]:
data[data_fields]

,t1p1_winrate,t1p2_winrate,t1p3_winrate,t1p4_winrate,t1p5_winrate,t2p1_winrate,t2p2_winrate,t2p3_winrate,t2p4_winrate,t2p5_winrate,t1_win
0,0.498571,0.509730,0.509872,0.515848,0.475056,0.501131,0.496120,0.488550,0.519051,0.515809,0
1,0.506253,0.481649,0.492001,0.486434,0.518533,0.528324,0.489466,0.468247,0.498443,0.512729,1
2,0.491480,0.482073,0.515809,0.471676,0.515556,0.517466,0.493931,0.493934,0.530648,0.496091,0
3,0.491480,0.495994,0.507958,0.455007,0.483320,0.501131,0.496120,0.501042,0.468247,0.512729,1
4,0.498571,0.478478,0.465220,0.472161,0.496091,0.491480,0.482073,0.472998,0.471676,0.468247,0
...,...,...,...,...,...,...,...,...,...,...,...
60151,0.479853,0.482073,0.529977,0.448276,0.502033,0.502269,0.507639,0.468247,0.455007,0.521420,0
60152,0.489466,0.495994,0.482688,0.513412,0.486415,0.500665,0.517481,0.507958,0.465220,0.521437,0
60153,0.520462,0.496120,0.497881,0.437414,0.490048,0.507645,0.502637,0.468247,0.465220,0.521420,1
60154,0.527956,0.509730,0.486415,0.515848,0.493812,0.528324,0.496120,0.509872,0.530648,0.475056,1


In [106]:
winrate_mat = data[data_fields].to_numpy()
np.save('data/winrate_mat.npy', winrate_mat)

In [109]:
mat = np.load('data/winrate_mat.npy')
mat.shape

(60156, 11)

###  Count by position

In [8]:
df = pd.read_csv('data/table3_n.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/table3_n.csv'

In [120]:
df

,champName,champId,ADC_appearances,ADC_winrate,ADC_wins,JUNGLE_appearances,JUNGLE_winrate,JUNGLE_wins,MIDDLE_appearances,MIDDLE_winrate,MIDDLE_wins,SUPPORT_appearances,SUPPORT_winrate,SUPPORT_wins,TOP_appearances,TOP_winrate,TOP_wins
0,Annie,1,26.0,0.576923,15.0,2.0,1.000000,2.0,856.0,0.556075,476.0,42.0,0.404762,17.0,82.0,0.536585,44.0
1,Olaf,2,27.0,0.518519,14.0,2798.0,0.515011,1441.0,41.0,0.365854,15.0,1.0,1.000000,1.0,600.0,0.515000,309.0
2,Galio,3,19.0,0.526316,10.0,8.0,0.375000,3.0,5680.0,0.530810,3015.0,732.0,0.513661,376.0,126.0,0.595238,75.0
3,TwistedFate,4,2.0,0.500000,1.0,1.0,0.000000,0.0,4164.0,0.500240,2083.0,5.0,0.400000,2.0,32.0,0.437500,14.0
4,XinZhao,5,NaN,NaN,NaN,454.0,0.517621,235.0,8.0,0.750000,6.0,NaN,NaN,NaN,6.0,0.500000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Draven,119,2852.0,0.515077,1469.0,1.0,0.000000,0.0,20.0,0.500000,10.0,NaN,NaN,NaN,10.0,0.400000,4.0
146,Lissandra,127,NaN,NaN,NaN,NaN,NaN,NaN,605.0,0.462810,280.0,2.0,0.500000,1.0,24.0,0.416667,10.0
147,Aatrox,266,2.0,0.500000,1.0,33.0,0.363636,12.0,182.0,0.434066,79.0,1.0,0.000000,0.0,1896.0,0.462025,876.0
148,Kalista,429,3594.0,0.500556,1799.0,4.0,0.250000,1.0,94.0,0.404255,38.0,NaN,NaN,NaN,177.0,0.457627,81.0


In [7]:
positions = ['ADC', 'JUNGLE', 'MIDDLE', 'SUPPORT', 'TOP']
table = []

for index, row in df.iterrows():
    record = [row[1], row[0]]

    adict = { f'{p}_appearances': row[f'{p}_appearances'] for p in positions if not pd.isna(row[f'{p}_appearances'])}
    alist = sorted(adict.items(), key=lambda x: x[1], reverse=True)
    
    
    for n in range(0, 2):
        if len(alist) == 1 and n >= 1:
            continue

        idx = list(row.index).index(alist[n][0])

        record.append(alist[n][0].strip('_appearances'))

        for i in range(idx, idx + 2):
            record.append(row[i])


    table.append(record)

KeyError: 'ADC_appearances'

In [122]:
df = pd.DataFrame(table, columns=['champId', 'champName', 'position', 'appearance', 'winrate', 'position', 'appearance', 'winrate'])

In [123]:
df.to_csv('data/top_two_positions.csv', index=False)